# UNet++ with Atrous Convolutions
This notebook contains the implementation of UNet++ with atrous (dilated) convolutions for retinal vessel segmentation.

In [ ]:
from tensorflow.keras import layers, models

def conv_block(x, kernelsize, filters, dropout, batchnorm=False):
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(x)
    if batchnorm:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)
    conv = layers.Conv2D(filters, (kernelsize, kernelsize), kernel_initializer='he_normal', padding="same")(conv)
    if batchnorm:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    return conv

def unetplusplus_atrous(input_shape, dropout=0.2, batchnorm=True):
    filters = [16, 32, 64, 128, 256]
    kernelsize = 3
    upsample_size = 2

    inputs = layers.Input(input_shape)

    # Downsampling
    dn_1 = conv_block(inputs, kernelsize, filters[0], dropout, batchnorm)
    pool_1 = layers.MaxPooling2D(pool_size=(2,2))(dn_1)
    dn_2 = conv_block(pool_1, kernelsize, filters[1], dropout, batchnorm)
    pool_2 = layers.MaxPooling2D(pool_size=(2,2))(dn_2)
    dn_3 = conv_block(pool_2, kernelsize, filters[2], dropout, batchnorm)
    pool_3 = layers.MaxPooling2D(pool_size=(2,2))(dn_3)
    dn_4 = conv_block(pool_3, kernelsize, filters[3], dropout, batchnorm)
    pool_4 = layers.MaxPooling2D(pool_size=(2,2))(dn_4)
    dn_5 = conv_block(pool_4, kernelsize, filters[4], dropout, batchnorm)

    # Atrous convolutions for multi-scale feature extraction
    atrous_1 = layers.Conv2D(filters[4], kernelsize, dilation_rate=1, padding='same')(dn_5)
    atrous_6 = layers.Conv2D(filters[4], kernelsize, dilation_rate=6, padding='same')(dn_5)
    atrous_12 = layers.Conv2D(filters[4], kernelsize, dilation_rate=12, padding='same')(dn_5)
    atrous_18 = layers.Conv2D(filters[4], kernelsize, dilation_rate=18, padding='same')(dn_5)
    atrous_combined = layers.Concatenate()([atrous_1, atrous_6, atrous_12, atrous_18])
    atrous_processed = conv_block(atrous_combined, kernelsize, filters[4], dropout, batchnorm)

    # Upsampling path
    up_5 = layers.UpSampling2D(size=(upsample_size, upsample_size))(atrous_processed)
    up_5 = layers.concatenate([up_5, dn_4], axis=3)
    up_conv_5 = conv_block(up_5, kernelsize, filters[3], dropout, batchnorm)
    up_4 = layers.UpSampling2D(size=(upsample_size, upsample_size))(up_conv_5)
    up_4 = layers.concatenate([up_4, dn_3], axis=3)
    up_conv_4 = conv_block(up_4, kernelsize, filters[2], dropout, batchnorm)
    up_3 = layers.UpSampling2D(size=(upsample_size, upsample_size))(up_conv_4)
    up_3 = layers.concatenate([up_3, dn_2], axis=3)
    up_conv_3 = conv_block(up_3, kernelsize, filters[1], dropout, batchnorm)
    up_2 = layers.UpSampling2D(size=(upsample_size, upsample_size))(up_conv_3)
    up_2 = layers.concatenate([up_2, dn_1], axis=3)
    up_conv_2 = conv_block(up_2, kernelsize, filters[0], dropout, batchnorm)

    conv_final = layers.Conv2D(1, kernel_size=(1,1))(up_conv_2)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    outputs = layers.Activation('sigmoid')(conv_final)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model